In [72]:

from rbot import Session
from rbot import Runner
from rbot import BinanceConfig
from rbot import BinanceMarket
from rbot import init_debug_log
from rbot import init_log
from rbot import OrderSide
from rbot import time_string
from rbot import OrderStatus
from rbot import SEC


In [73]:
MARGIN = 0.05

class MyAgent:
    def __init__(self):
        self.last_update = 0
        self.order_size = 0.001

    def on_clock(self, session, clock):
        if 0 < len(session.buy_orders):
            if session.current_time + SEC(60) < session.buy_orders[0].create_time:
                session.cancel_order(session.buy_orders[0].order_id)
                print("cancel buy order")
        
        if 0 < len(session.sell_orders):
            if session.current_time + SEC(60) < session.sell_orders[0].create_time:
                session.cancel_order(session.sell_orders[0].order_id)
                print("cancel sell order")

    def on_tick(self, session, side, price, size):
        if session.buy_order_amount == 0 and ((- self.order_size * session.buy_edge) < session.home_change):
            session.limit_order(OrderSide.Buy, session.buy_edge - MARGIN, self.order_size)
            #print("try make buy order", session.home_change, session.foreign_change)
            
        if session.sell_order_amount == 0 and ((- self.order_size) < session.foreign_change):
            session.limit_order(OrderSide.Sell, session.sell_edge + MARGIN, self.order_size)
            #print("try make sell order", session.home_change, session.foreign_change)
    
#    def on_update(self, session, updated_order):
#        print("on_update", updated_order)
        
    def on_account_update(self, session, account):
        print("account update: ", session.current_time, account)

    




In [74]:
market = BinanceMarket(BinanceConfig.BTCUSDT)

agent = MyAgent()
runner = Runner()

session = runner.back_test(market, agent, 60, 0, 0)

session

Done running agent


In [65]:
session.log_df[session.log_df["status"] == "Canceled"]

TypeError: cannot use `__getitem__` on DataFrame with item shape: (302,)
Series: 'status' [bool]
[
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	…
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
	false
] of type 'Series'

In [75]:
df = session.log_df.to_pandas(use_pyarrow_extension_array=True)
df


,symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,...,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
0,BTCUSDT,2023-10-14 00:00:07.497000,New,20231019T1515-0001BUY,20231019T1515-0001BUY,Buy,Limit,26861.95,0.001,0.00100,...,False,,0.0,0.0,0.000000,0.00000,-26.861950,0.00000,26.86195,0.00000
1,BTCUSDT,2023-10-14 00:00:07.497000,New,20231019T1515-0002SEL,20231019T1515-0002SEL,Sell,Limit,26862.06,0.001,0.00100,...,False,,0.0,0.0,0.000000,0.00000,0.000000,-0.00100,0.00000,0.00100
2,BTCUSDT,2023-10-14 00:00:07.497000,Filled,20231019T1515-0002SEL,20231019T1515-0002SEL,Sell,Limit,26862.06,0.001,0.00000,...,True,,0.0,0.0,26.864660,-0.00100,26.864660,-0.00100,0.00000,0.00100
3,BTCUSDT,2023-10-14 00:00:07.497000,PARTIALLY_FILLED,20231019T1515-0001BUY,20231019T1515-0001BUY,Buy,Limit,26861.95,0.001,0.00013,...,True,,0.0,0.0,-23.368679,0.00087,-23.368679,0.00087,26.86195,0.00000
4,BTCUSDT,2023-10-14 00:00:34.675000,New,20231019T1515-0003SEL,20231019T1515-0003SEL,Sell,Limit,26860.61,0.001,0.00100,...,False,,0.0,0.0,0.000000,0.00000,0.000000,-0.00100,0.00000,0.00100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,BTCUSDT,2023-10-15 13:10:27.036000,New,20231019T1515-0125SEL,20231019T1515-0125SEL,Sell,Limit,26840.04,0.001,0.00100,...,False,,0.0,0.0,0.000000,0.00000,0.000000,-0.00100,0.00000,0.00100
298,BTCUSDT,2023-10-15 13:10:27.036000,PARTIALLY_FILLED,20231019T1515-0125SEL,20231019T1515-0125SEL,Sell,Limit,26840.04,0.001,0.00075,...,True,,0.0,0.0,6.711390,-0.00025,6.711390,-0.00025,0.00000,0.00100
299,BTCUSDT,2023-10-15 13:10:27.036000,PARTIALLY_FILLED,20231019T1515-0125SEL,20231019T1515-0125SEL,Sell,Limit,26840.04,0.001,0.00053,...,True,,0.0,0.0,5.906043,-0.00022,5.906043,-0.00022,0.00000,-0.00022
300,BTCUSDT,2023-10-15 13:10:27.036000,PARTIALLY_FILLED,20231019T1515-0125SEL,20231019T1515-0125SEL,Sell,Limit,26840.04,0.001,0.00034,...,True,,0.0,0.0,5.100865,-0.00019,5.100865,-0.00019,0.00000,-0.00019


In [76]:
df.to_csv("backtest.csv")

In [67]:
df['status'].value_counts()

status
New                 125
Filled              124
PARTIALLY_FILLED     53
Name: count, dtype: int64[pyarrow]

In [68]:
df['foreign_change'].cumsum()

0      0.00000
1      0.00000
2     -0.00100
3     -0.00013
4     -0.00013
        ...   
297    0.00044
298    0.00019
299   -0.00003
300   -0.00022
301   -0.00122
Name: foreign_change, Length: 302, dtype: double[pyarrow]

In [69]:
session.log_df.sum()

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
str,datetime[μs],str,str,str,str,str,f64,f64,f64,str,datetime[μs],f64,f64,f64,f64,str,u32,str,f64,f64,f64,f64,f64,f64,f64,f64
null,null,null,null,null,null,null,8.1064e6,0.302,0.14718,null,null,4.7511e6,0.14992,4024.210756,0.0,null,177,null,0.0,0.0,33.062888,-0.00122,-1577.444622,-0.06622,2470.31255,0.11167


In [70]:
session.home_change + session.foreign_change * session.buy_edge

-1.4685043999999934

In [71]:
len(session.log_df)

302